In [1]:
from __future__ import division
import pandas as pd
from collections import Counter
from itertools import chain, groupby
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import numpy as np
import random

In [ ]:
class LogisticRegression:
    
    def sigmoid_function():
        

In [11]:
class NaiveBayesClassifier:
    
    def __init__(self, df, output_col_name, features):
        self.df = df
        self.conditional_probabilites = {}
        self.prior_probabilites = {}
        self.output = output_col_name
        self.features = features
        
    def calculate_prior_probabilites(self):
        groups = self.df.groupby(self.output).groups
        for x in groups:
            self.prior_probabilites[x] = len(groups[x])/len(df)
    
    def calculate_conditional_probabilites(self):
        for x in self.features:
            if x != self.output:
                self.conditional_probabilites[x] = self.df.groupby(self.output)[x].value_counts()/self.df.groupby(self.output)[x].count()
                
    def train(self):
        self.calculate_prior_probabilites()
        self.calculate_conditional_probabilites()
    
    def predict(self, row):
        posterior_prob = {}
        
        # Initialize the posterior probability as same as the priors
        for x in self.prior_probabilites:
            posterior_prob[x] = self.prior_probabilites[x]
        
        for label in posterior_prob:
            for x in self.features:
                if x != self.output:
                    if label in self.conditional_probabilites[x] and row[x] in self.conditional_probabilites[x].get(label):
                        posterior_prob[label] *= self.conditional_probabilites[x].get(label).get(row[x])
                    else:
                        posterior_prob[label] = 0
                        break
        return max(posterior_prob, key=posterior_prob.get)
        
        
        

In [12]:
dataPath = './DataSets/breast-cancer-wisconsin.data.txt'
columns = ['id', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'output']
features = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'output']
df = pd.read_csv(dataPath, names=columns)
df = df.replace('?',np.NAN)
df.isnull().sum()

imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df[features])
imputed_data = imr.transform(df[features]);

In [13]:
df['output'] = df['output'].apply(lambda x : 1 if x == 2 else -1)

In [14]:
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df[features], df['output'], 
                                                                test_size=0.33, random_state=42)

In [33]:
fractions = [0.01, 0.02, 0.03, 0.125, 0.625, 1]
training_data_size = []
for fraction in fractions:
    df_train = df_X_train.sample(frac=fraction)
    training_data_size.append(len(df_train))
    nbclassifier = NaiveBayesClassifier(df=df_train, output_col_name='output', features=features)
    nbclassifier.train()
    df_X_test['prediction'] = df_X_test.apply(lambda row : nbclassifier.predict(row), axis=1)
    accuracy_results = pd.crosstab(df_X_test['prediction'], df_y_test)
    
    misclassifcation = 0;
    
    if 1 in accuracy_results:
        if -1 in accuracy_results[1]:
            misclassifcation += accuracy_results[1][-1]
        
    
    if -1 in accuracy_results:
        if 1 in accuracy_results[-1]:
            misclassifcation += accuracy_results[-1][1]
    
    print misclassifcation

77
77
72
35
7
7
